In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import concurrent
from time import sleep

O que este notebook ira realizar:
-------------------------------

1. Acessar o site da olx utilizando a biblioteca **BeautifulSoup**, ecnontrar a lista de preços e salvar cada item em uma lista de "Tags"
2. Encontrar os valores (municipio, estado , categoria, titulo, preco, link) de cada anuncio na lista
3. Salvar em um DataFrame do Pandas
4. Acessar cada um dos links de cada anuncio no DataFrame, no campo **href** e retira dele o CEP de cada anuncio criando um dicionario neste formato {"href":"cep"} **(Esta parte do codigo precisa ser aprimorada atualmente demora em torno de 5 à 7min para executar)**
5. Adicionar o dicionario de CEP ao dataframe de anuncios utilizando a função _.map_ do Pandas 
6. Salvar o DataFrame resultante em uma planilha Excel

In [2]:
labels = ['local', 'categoria', 'titulo', 'preco', 'href', 'data']
cep_dict = {}
url_leste = "http://sp.olx.com.br/sao-paulo-e-regiao/zona-leste/imoveis?o="
url_oeste = 'http://sp.olx.com.br/sao-paulo-e-regiao/zona-oeste/imoveis?o='
url_centro = 'http://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis?o='
url_norte = 'http://sp.olx.com.br/sao-paulo-e-regiao/zona-norte/imoveis?o='
url_sul = 'http://sp.olx.com.br/sao-paulo-e-regiao/zona-sul/imoveis?o='

In [3]:
def retornaSoupOLX(url, num):
    li_list = []
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    for iter in range(num):
        sleep(1)
        r = requests.get(str(url)+str(iter), headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        for ultag in soup.find_all('ul', {'class': 'list', 'id':'main-ad-list' }):
            for litag in ultag.find_all(lambda tag: tag.name == 'li' and tag.get('class') == ['item']):
                li_list.append(litag)
    return li_list

In [4]:
def tiraEspecialString(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '')

In [5]:
def tiraEspecialFloat(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '').replace(',', '.')

In [6]:
%%time

def criaListaValores(li_list):
    valores = []
    for elements in li_list:
        x = BeautifulSoup(str(elements), 'html.parser')
        #encontra a parte onde estão as informaçoes no html
        info = x.find('div', {'class':'col-2'})
        #encontra a string onde estão o municipio e estado separados por virgula
        cidade_estado = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-region'}))
        #encontra a categoria do anuncio
        categoria = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-category'}))
        #encontra o titulo do anuncio
        titulo = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-1 mb5px'}).find('h3', {'class':'OLXad-list-title mb5px'}))
        #encontra o html do preço
        preco = x.find('div', {'class':'col-3'})
        #encontra a data
        data = x.find('div', {'class':'col-4'}).find('p', {'class':'text mb5px'}).text
        
        #trata o html do preço verificando se existe ou não
        if preco.find('p', {'class':'OLXad-list-price'}):
            preco = tiraEspecialFloat(x.find('div', {'class':'col-3'}).find('p', {'class':'OLXad-list-price'}))
        else:
            preco = 0.0
        #retorna a tag href "link do anuncio"
        href=x.find('a', {'class':'OLXad-list-link'})
        
        #retorna o conteudo do atributo href na tag href
        href = href['href']
        
        valores.append([cidade_estado, categoria, titulo, preco, href, data])
    return valores
        

Wall time: 0 ns


In [7]:
def findCEP(url):
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    regx = re.compile('\d{5}-\d{3}')
    r = requests.get(url, headers = headers)
    sleep(1)
    soup = BeautifulSoup(r.content, 'html.parser')
    cep = soup.find(lambda tag: tag.name == 'strong' and tag.get('class') == ['description'] and regx.match(tag.text.strip(' \t\n\r')) is not None)
    if cep:
        cep_dict.update({url:cep.text.strip(' \t\n\r')})
    else:
        cep_dict.update({url:''})

In [8]:
%%time
#cria as Soups de cada região
soup_leste = retornaSoupOLX(url_leste, 100)
soup_sul = retornaSoupOLX(url_sul,100)
soup_norte = retornaSoupOLX(url_norte, 100)
soup_oeste = retornaSoupOLX(url_oeste, 100)
soup_centro = retornaSoupOLX(url_centro, 100)

Wall time: 15min 57s


In [9]:
%%time
#cria lista com os valores separados de cada região
valores_leste = criaListaValores(soup_leste)
valores_sul = criaListaValores(soup_sul)
valores_norte = criaListaValores(soup_norte)
valores_oeste = criaListaValores(soup_oeste)
valores_centro = criaListaValores(soup_centro)

Wall time: 3min 23s


In [10]:
%%time
#cria o dataframe de cada uma das regiões
df_norte = pd.DataFrame(valores_norte, columns=labels)
df_sul = pd.DataFrame(valores_sul, columns=labels)
df_leste = pd.DataFrame(valores_leste, columns=labels)
df_oeste = pd.DataFrame(valores_oeste, columns=labels)
df_centro = pd.DataFrame(valores_centro, columns=labels)

Wall time: 34 ms


In [11]:
#cria uma coluna com a Zona de cada dataframe
df_norte['zona'] = 'Norte'
df_sul['zona'] = 'Sul'
df_leste['zona'] = 'Leste'
df_oeste['zona'] = 'Oeste'
df_centro['zona'] = 'Centro'

In [12]:
#criar uma lista de DataFrames e juntalos em apenas um
df = pd.DataFrame()
lista_df_zona = [
    df_norte,
    df_sul,
    df_leste,
    df_oeste,
    df_centro
]
df = df.append(lista_df_zona, ignore_index=True)

In [13]:
#Retira as categorias que estão mal formatadas e cria uma coluna á parte para os anuncios profissionais
df['categoria'] = df['categoria'].str.replace('- Anúncio Profissional', 'Profissional')
df['categoria'] = df['categoria'].str.replace('Profissional', '- Profissional')
df['categoria'], df['profissional'] = df['categoria'].str.split('-', 1).str
df['profissional']=df.profissional.replace(' Profissional', True)
df['profissional']=df.profissional.fillna(False)

In [14]:
#selecionar casas com preços diferente de 0
df_casa = df[df['categoria'] == 'Casas']
df_casa['preco'] = df_casa['preco'].astype(float)
df_casa = df_casa[df_casa['preco']!=0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
#Cria uma lista com  os links de cada anuncio
list_href = list(df_casa['href'])

In [16]:
%%time
#Usa Pools para realizar a busca de cep e adicionar ao dicionario 'dict_cep'
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as e:
     e.map(findCEP, list_href)

Wall time: 10min 25s


In [17]:
#Cira um campo CEP no DataFrame juntando o dicionario criado anteriormente
df_casa['cep'] = df_casa['href'].map(cep_dict)

In [18]:
#Aqui dropa o antigo local e separa o 'São Paulo, Bairro' em duas colunas
df_casa = pd.concat([df_casa , df_casa['local'].str.split(',', n=1, expand=True)], axis =1)
df_casa.drop(['local', 0], axis=1, inplace=True)

In [19]:
#Aqui dropa o antigo local e separa o 'São Paulo, Bairro' em duas colunas
df = pd.concat([df , df['local'].str.split(',', n=1, expand=True)], axis =1)
df.drop(['local', 0], axis=1, inplace=True)

In [20]:
#Arruma as colunas criadas anteriormente
df.columns = ['categoria', 'titulo', 'preco', 'href', 'data', 'zona','profissional', 'bairro']

In [21]:
#Arruma as colunas criadas anteriormente
df_casa.columns = ['categoria', 'titulo', 'preco', 'href', 'data', 'zona','profissional', 'cep', 'bairro']

In [22]:
#Arruma a coluna bairro pois há um espaço como primeiro caractere em todos os registros
df_casa['bairro'] = df_casa['bairro'].str.replace(' ', '', n=1)
df['bairro'] = df['bairro'].str.replace(' ', '', n=1)

In [23]:
df_casa.reset_index(drop=True, inplace=True)
df_casa.head()

,categoria,titulo,preco,href,data,zona,profissional,cep,bairro
0,Casas,Casa,590000.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,02311-100,Vila Mazzei
1,Casas,Sobrado COD L563,2800.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,02378-070,Horto Florestal
2,Casas,Sobrado COD L564,2950.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,02080-010,Vila Isolina Mazzei
3,Casas,Cod TR 398 - Vila Mazzei,650.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,02312-170,Vila Nova Mazzei
4,Casas,Cod TR 420 - Sobrado,1100.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,02281-224,Cj Hab J Rural


In [24]:
%%time
df_casa.loc[:,df_casa.columns != 'cep'].to_excel('../1-OLX_WebScrap/1_OLX_CASA_SEM_CEP.xlsx')
df_casa.to_excel('../1-OLX_WebScrap/1_OLX_CASA_COM_CEP.xlsx')
df.to_excel('../1-OLX_WebScrap/1_OLX_TODOS_SEM_CEP.xlsx')

Wall time: 28.5 s


In [26]:
df.head()

,categoria,titulo,preco,href,data,zona,profissional,bairro
0,Apartamentos,"Oportunidade, Apartamento 02 dormitórios, 02 b...",319000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,False,Parada Inglesa
1,Casas,Casa,590000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,Vila Mazzei
2,Apartamentos,Apartamento,1600,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,Santana
3,Apartamentos,Apartamento,1700,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,Vila Guilherme
4,Apartamentos,Apartamento,1400,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Hoje,Norte,True,Tucuruvi
